In [26]:
import json

In [27]:
# Prompt + agent_name
rules_prompt = """You are playing a game called the Werewolf with some other players. This game is based on text conversations. Here are
the game rules: Roles: The moderator is also host, he organised this game and you need to answer his instructions correctly.
Don't talk with the moderator. There are five roles in the game, werewolf, villager, seer, guard and witch. There are two
alternate phases in this game, daytime and dark. When it's dark: Your talking content with moderator is confidential. You
needn't worry about other players and moderator knowing what you say and do. No need to worry about suspicions from
others during the night. If you are werewolf, you can know what your teammates want to kill and you should vote one player
to kill based on your analysis. Player who receives the most votes after all werewolves voting will be killed. No one will be
killed if there is no consensus! If you are witch, you have a bottle of antidote that can save a player targeted by werewolves
after dark, and a bottle of poison that can poison a player after dark. Both poison and antidote can be used only once. If you
are seer, you can verify whether a player is a werewolf every night, which is a very important thing. If you are guard, you
can protect a player every night to prevent the player from being killed by werewolves, but guard cannot resist the witch's
poison and guard cannot protect the same player on two consecutive nights. Villagers can't do anything at night. During the
daytime: you discuss with all players including your enemies. At the end of the discussion, players vote to eliminate one
player they suspect of being a werewolf. The player with the most votes will be eliminated. The moderator will tell who is
killed, otherwise there is no one killed. Note that villager, seer, guard and witch are all in villager side, they have the same
objective. Objectives: If you are werewolf, your goal is to cooperate with other werewolves to kill all players who are not
werewolves at last. If you are not werewolf, you need to kill all werewolves with your partner once you find out that certain
players are suspicious to be werewolves. This could greatly improve your chances of winning, although it is somewhat risky.If
one player is killed, he can't do anything anymore and will be out of the game. Tips: To complete the objective: During
night, you should analyze and use your ability correctly. During daytime, you need to reason carefully about the roles of other
players and be careful not to reveal your own role casually unless you're cheating other players. Only give the player's name
when making a decision/voting, and don't generate other players' conversation.Reasoning based on facts you have observed
and you cannot perceive information (such as acoustic info) other than text. 
You're playing with 6 other players. Do not pretend you are other players or the moderator. Always end your response with
'<EOS>'. You are Player: """

In [28]:
#For this we require the game to fill in who is the werewolf who are living, valid players. 
#Werewolfs + prompt
initial_werewolf_prompt = """Werewolves, please open your eyes! I secrecly tell you that Player 1, Player 2 are all of
the 2 werewolves! Keep in mind you are teammates. The rest players are not werewolves."""

In [29]:
day_start_prompt = """The sun rose. Everyone woke up except those who had been killed."""
day_discuss_prompt = """Now freely talk about roles of other players with each other based on your observation and reflection with
few sentences. Decide whether to reveal your identity based on your reflection. """

In [30]:
#player name + prompt + [living players + pass]
day_vote_prompt = """ are asked to choose which of the players should be voted for killing based on the
discussion? Don't mention your role. For example: I vote to kill Player... You can only choose one from the following living options please: """

In [31]:
first_night_prompt = """It's dark, everyone close your eyes. I will talk with you/your team secretly at night."""
# This is a slight diversion from Xu et.al because we have both a discussion and a voting phase at night. 
night_discuss_prompt = """ Werewolves, please open your eyes! Now discuss with your teammates which of the
players should be killed tonight. """

In [32]:
#Prompts for votes to perform night actions. 

#Werewolf names + prompt + [living townsfolk + pass].
# This is a slight diversion from Xu et.al because we have both a discussion and a voting phase at night. 
night_vote_prompt_werewolf = """ I secretly tell you that you are all of the 2 werewolves! Keep in mind you are teammates. The rest players are not werewolves. 
Please vote which of the players should be killed tonight. You only choose one from the following living options please: """
#Guard name + prompt + [living players + pass].
night_vote_prompt_guard = """You guard, please open your eyes! Now tell me who you protect tonight? You only
choose one from the following living options please: """
#Witch name + prompt + [living players + pass - witch name].
night_vote_prompt_witch = """ You witch, please open your eyes! You have a bottle of poison, who are you going to kill
tonight? Choose one from the following living options: """
#Seer name + prompt + [living players + pass - Seer name].
night_vote_prompt_seer = """ You seer, please open your eyes! Who are you going to verify its identity tonight?
Choose only one from the following living options: """

In [33]:
werewolf_prompt = "You are werewolf!"
townsfolk_prompt = "You are villager!"
seer_prompt = "You are seer!"
guard_prompt = "You are guard!"
witch_prompt = "You are witch!"
#One game mode had an additional hunter roll not in Xu et.al
hunter_prompt = "You are villager!"

In [34]:
#player name + prompt
seer_werewolf_prompt =  " is a werewolf!"
seer_townsfolk_prompt = " is a villager!"
seer_seer_prompt = " is a seer!"
seer_guard_prompt = " is a guard!"
seer_witch_prompt = " is a witch!"
seer_hunter_prompt = " is a villager!"

In [35]:
#List of dead + prompt
who_died = " died last night!"
nobody_died = "It was a peaceful night and no one died!"

In [36]:
#Now to make the Json itself. 
dict_for_json = {
    "rules_prompt":rules_prompt,
    "initial_werewolf_prompt":initial_werewolf_prompt,
    "day_start_prompt":day_start_prompt,
    "day_discuss_prompt":day_discuss_prompt,
    "day_vote_prompt":day_vote_prompt,
    "first_night_prompt":first_night_prompt,
    "night_discuss_prompt":night_discuss_prompt,
    "night_vote_prompt_werewolf":night_vote_prompt_werewolf,
    "night_vote_prompt_guard":night_vote_prompt_guard,
    "night_vote_prompt_witch":night_vote_prompt_witch,
    "night_vote_prompt_seer":night_vote_prompt_seer,
    "werewolf_prompt":werewolf_prompt,
    "townsfolk_prompt":townsfolk_prompt,
    "seer_prompt":seer_prompt,
    "guard_prompt":guard_prompt,
    "witch_prompt":witch_prompt,
    "hunter_prompt":hunter_prompt,
    "seer_werewolf_prompt":seer_werewolf_prompt,
    "seer_townsfolk_prompt":seer_townsfolk_prompt,
    "seer_seer_prompt":seer_seer_prompt,
    "seer_guard_prompt":seer_guard_prompt,
    "seer_witch_prompt":seer_witch_prompt,
    "seer_hunter_prompt":seer_hunter_prompt,
    "who_died":who_died,
    "nobody_died":nobody_died
}

In [37]:
with open("new_prompt.json", "w") as outfile: 
    json.dump(dict_for_json, outfile)